In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
from account import Binance
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
import cvxpy as cp
from utils.logging import get_logger
from utils.data_helper import *
from utils.db import *
from strategy_v3.Strategy import ExchangeArbitrageStrategy

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 50)
pd.options.display.float_format = "{:,.4f}".format
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Expermental Arbitrage strategy in Binance

1. Get bid/ask price for all active currency pairs in Binance and presents in a matrix $Q$

2. Transform the price to negative log price (Given converison from currency A->B->C = log(p1*p2) ~= log(p1) + log(p2))

3. Solve the optimization follow classic Traveling Salesmen Problem (TSP), but removing the constraints that all nodes needs to be visited once

- We want to find a closed loop where the sum of path values are negative

- Input $X$ is the nxn binary matrix (n is number of assets), 1 represents trade from currency x -> y

- Minimize $X$ dot $Q$

Reference: https://nbviewer.org/github/rcroessmann/sharing_public/blob/master/arbitrage_identification.ipynb


In [3]:
strategy = ExchangeArbitrageStrategy(zero_fees=True)
strategy.set_strategy_id("qa")

In [6]:
strategy.load_data()

In [7]:
logger = get_logger(strategy.__str__())
logger.info(f"BTC -> ETH: {strategy.quote_matrix[strategy.assets.index('BTC')][strategy.assets.index('ETH')]}")
logger.info(f"ETH -> BTC: {strategy.quote_matrix[strategy.assets.index('ETH')][strategy.assets.index('BTC')]}")
logger.info(f"BTC -> ETH: {strategy.quote_matrix_w_fee[strategy.assets.index('BTC')][strategy.assets.index('ETH')]}")
logger.info(f"ETH -> BTC: {strategy.quote_matrix_w_fee[strategy.assets.index('ETH')][strategy.assets.index('BTC')]}")

2025-01-15 00:48:55,509 - qa - INFO - BTC -> ETH: 29.976019184652277
2025-01-15 00:48:55,510 - qa - INFO - ETH -> BTC: 0.03335
2025-01-15 00:48:55,510 - qa - INFO - BTC -> ETH: 29.976019184652277
2025-01-15 00:48:55,510 - qa - INFO - ETH -> BTC: 0.03335


# Find the optimal paths and generate trades

In [8]:
strategy.optimize()

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Jan 15 12:48:56 AM: Your problem has 162409 variables, 4 constraints, and 0 parameters.
(CVXPY) Jan 15 12:48:56 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 15 12:48:56 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 15 12:48:56 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 15 12:48:56 AM: Compiling problem (target solver=SCIPY).
(CVXPY) Jan 15 12:48:56 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffi

2025-01-15 00:49:00,055 - qa - INFO - CVXPY - Status: optimal
2025-01-15 00:49:00,055 - qa - INFO - CVSPY - Optimal value: -0.0012077600728244153
2025-01-15 00:49:00,058 - qa - INFO - Total PNL: -1.0642%
2025-01-15 00:49:00,241 - qa - INFO - 
+---------+--------------+------------+---------+
|   group |   gross_pnl% |   net_pnl% |   count |
|---------+--------------+------------+---------|
|       1 |   0.00417317 |  -0.295539 |       3 |
|       2 |   0.116671   |  -0.770926 |       8 |
+---------+--------------+------------+---------+


# Trade the arbitrage pair(s)
- Trade the pairs with highest pnl
- The arbitrage loop starts with one existing currency in current portfolios

- commission are included in quote quantity?

In [10]:
trade_currency = {
    'USDT': 50,
    'ETH':  0.01539456264,
    'BTC': 0.00052979531,
}

In [11]:
client = Binance().client
balance = client.get_account()
balance = pd.DataFrame(balance['balances'])
balance['free'] = balance['free'].astype(float)
balance['locked'] = balance['locked'].astype(float)
balance = balance[balance['free'] > 0]
balance

,asset,free,locked
0,BTC,0.0701,0.0000
2,ETH,1.8639,0.0000
11,USDT,"9,775.2708","2,416.7007"
86,ADA,0.0734,0.0000
88,XLM,99.9000,0.0000
143,DENT,0.7540,0.0000
197,EUR,0.0938,0.0000
202,TRY,28.5318,0.0000
227,JPY,4.6434,0.0000
239,SOL,4.6596,0.0000


In [12]:
# Trade the First Group For Now
df_trades = strategy.df_trades
df_pnl = strategy.df_pnl
trades = df_trades[df_trades['group'] == df_pnl.index[0]]

# reorder the trades to start with one of the trade currency
start_ccys = [x for x in trade_currency if x in trades['from_asset'].to_list()]
if len(start_ccys) == 0:
    logger.info('No trade currency found')
    raise SystemExit()

start_ccy = start_ccys[0]
start_order = trades[trades['from_asset'] == start_ccy].iloc[0]['order']
trades['order'] = (trades['order'] - start_order) % len(trades) + 1
trades = trades.sort_values('order').reset_index(drop=True)

# sanity check
assert trades.iloc[0]['from_asset'] == start_ccy
assert trades.iloc[-1]['to_asset'] == start_ccy
trades

,from_asset,to_asset,mkt_price,fee,group,order,mkt_price_w_fee,symbol,status,baseAsset,quoteAsset,stepSize,tickSize,qty_decimal,price_decimal,bidPrice,bidQty,askPrice,askQty,makerCommission,takerCommission,side,count,price_time,zero_fees
0,ETH,AVAX,88.4173,0.0010,1,1,88.3289,AVAXETH,TRADING,AVAX,ETH,0.01000000,0.00001000,2,5,0.01130000,26.77000000,0.01131000,4.38000000,0.0010,0.0010,BUY,1,2025-01-15 00:48:54.894024+08:00,True
1,AVAX,FDUSD,36.0700,0.0010,1,2,36.0339,AVAXFDUSD,TRADING,AVAX,FDUSD,0.01000000,0.01000000,2,2,36.07000000,52.28000000,36.08000000,38.07000000,0.0000,0.0010,SELL,1,2025-01-15 00:48:54.894024+08:00,True
2,FDUSD,ETH,0.0003,0.0010,1,3,0.0003,ETHFDUSD,TRADING,ETH,FDUSD,0.00010000,0.01000000,4,2,3188.93000000,0.11940000,3189.08000000,0.78430000,0.0000,0.0010,BUY,1,2025-01-15 00:48:54.894024+08:00,True


In [ ]:
trade_orders = []

# execute the trades via market orders
for i, row in trades.iterrows():
    symbol = row['symbol']
    side = row['side']
    from_asset = row['from_asset']
    to_asset = row['to_asset']    

    baseAsset = row['baseAsset']
    baseAsset_decimal = row['qty_decimal']
    quoteAsset_decimal = row['price_decimal']
    
    # price is from_asset to to_asset, not the quoted price
    mkt_price = float(row['mkt_price'])    

    # First trade are based on pre-set amounts
    # qty is based on from_asset
    if i == 0:            
        from_asset_qty = trade_currency[from_asset]        
    
    order_params = {
        'symbol': symbol,
        'side': side,
        'type':'MARKET'
    }

    if from_asset == baseAsset:
        order_params['quantity'] = round_down(from_asset_qty, baseAsset_decimal)
    else:
        order_params['quoteOrderQty'] = round_down(from_asset_qty, quoteAsset_decimal)
    
    order = client.create_order(**order_params)                    
    logger.info(f'Created Market Order: {order_params}')
    
    if order['status'] == 'FILLED':   

        # find executed quantity for to_asset with current trade, which is from_asset for next trade     
        from_asset_qty = 0
        for fill in order['fills']:
            if to_asset == baseAsset:
                from_asset_qty += float(fill['qty'])
            else:
                from_asset_qty += float(fill['qty']) * float(fill['price'])

            # subtract out comission to get net received quantity
            if 'commission' in fill and to_asset == fill['commissionAsset']:
                from_asset_qty -= float(fill['commission'])

        trade_orders.append(order)
        
    else:
        raise Exception('order not filled yet!')    

2025-01-12 02:40:47,164 - Arbitrage - INFO - Created Market Order: {'symbol': 'DENTUSDT', 'side': 'BUY', 'type': 'MARKET', 'quoteOrderQty': 50.0}
2025-01-12 02:40:47,279 - Arbitrage - INFO - Created Market Order: {'symbol': 'DENTETH', 'side': 'SELL', 'type': 'MARKET', 'quantity': 39206.0}
2025-01-12 02:40:47,481 - Arbitrage - INFO - Created Market Order: {'symbol': 'ETHUSDT', 'side': 'SELL', 'type': 'MARKET', 'quantity': 0.0148}


In [ ]:
df_orders = pd.DataFrame(trade_orders).drop(columns=['fills'])
df_orders = df_orders.rename(columns={'side': 'order_side'})

df_fills = df_fills = pd.DataFrame([dict({'symbol': x['symbol']},**y) for x in trade_orders for y in x['fills']])
df_fills = df_fills.rename(columns={'price': 'fill_price', 'qty': 'fill_qty'})
df_fills['fill_price'] = df_fills['fill_price'].astype(float)
df_fills['fill_qty'] = df_fills['fill_qty'].astype(float)
df_fills['commission'] = df_fills['commission'].astype(float)

df_orders = pd.merge(df_orders, df_fills, how='left', on=['symbol'], validate='1:m')
df_orders = df_orders.rename(columns={'status': 'fill_status'})
df_orders = pd.merge(trades, df_orders, how='left', on=['symbol'], validate='1:m')

df_orders['from_asset_qty'] = df_orders.apply(lambda x: x['fill_qty'] if x['from_asset'] == x['baseAsset'] else x['fill_qty'] * x['fill_price'], axis=1)
df_orders['to_asset_gross_qty'] = df_orders.apply(lambda x: x['fill_qty'] if x['to_asset'] == x['baseAsset'] else x['fill_qty'] * x['fill_price'], axis=1)
df_orders['to_asset_comms_qty'] = df_orders.apply(lambda x: x['commission'] if x['to_asset'] == x['commissionAsset'] else 0, axis=1)
df_orders['to_asset_qty'] = df_orders['to_asset_gross_qty'] + df_orders['to_asset_comms_qty']
df_orders

,from_asset,to_asset,mkt_price,fee,group,order,mkt_price_w_fee,symbol,status,baseAsset,quoteAsset,stepSize,tickSize,qty_decimal,price_decimal,bidPrice,bidQty,askPrice,askQty,makerCommission,takerCommission,side,orderId,orderListId,clientOrderId,transactTime,price,origQty,executedQty,origQuoteOrderQty,cummulativeQuoteQty,fill_status,timeInForce,type,order_side,workingTime,selfTradePreventionMode,fill_price,fill_qty,commission,commissionAsset,tradeId,from_asset_qty,to_asset_gross_qty,to_asset_comms_qty,to_asset_qty
0,USDT,DENT,784.9294,0.0000,1,1,784.9294,DENTUSDT,TRADING,DENT,USDT,1.00000000,0.00000100,0,6,0.00127200,4327988.00000000,0.00127400,2455314.00000000,0.0010,0.0010,BUY,879587216,-1,0Sd5As75OW7ftlHkBLXfJy,1736620847172,0.00000000,39246.00000000,39246.00000000,50.00000000,49.99940400,FILLED,GTC,MARKET,BUY,1736620847172,EXPIRE_MAKER,0.0013,"11,015.0000",11.0150,DENT,82309883,14.0331,"11,015.0000",11.0150,"11,026.0150"
1,USDT,DENT,784.9294,0.0000,1,1,784.9294,DENTUSDT,TRADING,DENT,USDT,1.00000000,0.00000100,0,6,0.00127200,4327988.00000000,0.00127400,2455314.00000000,0.0010,0.0010,BUY,879587216,-1,0Sd5As75OW7ftlHkBLXfJy,1736620847172,0.00000000,39246.00000000,39246.00000000,50.00000000,49.99940400,FILLED,GTC,MARKET,BUY,1736620847172,EXPIRE_MAKER,0.0013,"28,231.0000",28.2310,DENT,82309884,35.9663,"28,231.0000",28.2310,"28,259.2310"
2,DENT,ETH,0.0000,0.0000,1,2,0.0000,DENTETH,TRADING,DENT,ETH,1.00000000,0.00000001,0,8,0.00000039,4953406.00000000,0.00000040,14616354.00000000,0.0010,0.0010,SELL,80902951,-1,oBvwaEkfMTGBo3SvNJCKSU,1736620847255,0.00000000,39206.00000000,39206.00000000,0.00000000,0.01489828,FILLED,GTC,MARKET,SELL,1736620847255,EXPIRE_MAKER,0.0000,"39,206.0000",0.0000,ETH,7735913,"39,206.0000",0.0149,0.0000,0.0149
3,ETH,USDT,"3,268.3200",0.0000,1,3,"3,268.3200",ETHUSDT,TRADING,ETH,USDT,0.00010000,0.01000000,4,2,3268.32000000,2.45880000,3268.33000000,84.01760000,0.0010,0.0010,SELL,23561677040,-1,bnk2pvCr972f9JSXl5lJoA,1736620847417,0.00000000,0.01480000,0.01480000,0.00000000,48.36640000,FILLED,GTC,MARKET,SELL,1736620847417,EXPIRE_MAKER,"3,268.0000",0.0148,0.0484,USDT,2036798668,0.0148,48.3664,0.0484,48.4148


In [ ]:
df_orders_agg = df_orders.groupby(['group', 'order', 'from_asset']).sum(numeric_only=True).reset_index()
df_orders_agg = df_orders_agg.groupby(['group']).agg({'from_asset': 'first', 'from_asset_qty': 'first', 'to_asset_qty': 'last'})
df_orders_agg['net_qty'] = df_orders_agg['to_asset_qty'] - df_orders_agg['from_asset_qty']
df_orders_agg

,from_asset,from_asset_qty,to_asset_qty,net_qty
group,,,,
1,USDT,49.9994,48.4148,-1.5846
